In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [146]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 11

5


In [147]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=572, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=91, max_features='sqrt',\n                           min_samples_leaf=17, n_estimators=140,\n                           random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  "LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.99330891532001

In [148]:
df = pd.read_csv("../data/ufc-master.csv")

In [149]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [150]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [151]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [152]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4187
4187
4187


In [153]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Alexander Hernandez,Drew Dober,100,-120,100.000000,83.333333,2020-05-13,"Jacksonville, Florida, USA",USA,Blue,...,0.0,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
201,Ricky Simon,Ray Borg,-170,150,58.823529,150.000000,2020-05-13,"Jacksonville, Florida, USA",USA,Red,...,2.0,0.63,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0
202,Andrei Arlovski,Philipe Lins,130,-150,130.000000,66.666667,2020-05-13,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
203,Michael Johnson,Thiago Moises,-110,-110,90.909091,90.909091,2020-05-13,"Jacksonville, Florida, USA",USA,Blue,...,3.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
204,Sijara Eubanks,Sarah Moras,-370,310,27.027027,310.000000,2020-05-13,"Jacksonville, Florida, USA",USA,Red,...,1.0,1.00,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4382,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4383,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4384,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4385,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Colby Covington,Tyron Woodley,-335,260,29.850746,260.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Khamzat Chimaev,Gerald Meerschaert,-400,300,25.000000,300.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Johnny Walker,Ryan Spann,-125,100,80.000000,100.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Mackenzie Dern,Randa Markos,-177,140,56.497175,140.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Kevin Holland,Darren Stewart,-265,205,37.735849,205.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Nate Landwehr,Darren Elkins,100,-120,100.000000,83.333333,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
196,Cortney Casey,Mara Romero Borella,-185,160,54.054054,160.000000,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,2.0,0.00,0.5,2.0,0.0,0.0,0.0,0.0,0.0,0
197,Rodrigo Nascimento,Don'Tale Mayes,-115,-105,86.956522,95.238095,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,1.00,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0
198,Anthony Smith,Glover Teixeira,-175,155,57.142857,155.000000,2020-05-13,"Jacksonville, Florida, USA",USA,Blue,...,2.0,0.33,0.5,0.0,0.0,0.0,5.0,0.0,0.0,1


In [154]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


C:\Users\Matt-Predator\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [155]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Colby Covington,Tyron Woodley,-335,260,29.850746,260.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Khamzat Chimaev,Gerald Meerschaert,-400,300,25.000000,300.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Johnny Walker,Ryan Spann,-125,100,80.000000,100.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Mackenzie Dern,Randa Markos,-177,140,56.497175,140.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Kevin Holland,Darren Stewart,-265,205,37.735849,205.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Nate Landwehr,Darren Elkins,100,-120,100.000000,83.333333,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
196,Cortney Casey,Mara Romero Borella,-185,160,54.054054,160.000000,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,2.0,0.00,0.5,2.0,0.0,0.0,0.0,0.0,0.0,0
197,Rodrigo Nascimento,Don'Tale Mayes,-115,-105,86.956522,95.238095,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,1.00,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0
198,Anthony Smith,Glover Teixeira,-175,155,57.142857,155.000000,2020-05-13,"Jacksonville, Florida, USA",USA,Blue,...,2.0,0.33,0.5,0.0,0.0,0.0,5.0,0.0,0.0,1


In [156]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [157]:
models[1][model_num]

'GaussianNB(var_smoothing=1e-12)'

In [158]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [159]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [160]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [161]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [162]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_9_10_20
GaussianNB(priors=None, var_smoothing=1e-12)
['B_Weight_lbs', 'B_Flyweight_rank', 'B_avg_TD_landed', 'B_Light Heavyweight_rank', 'B_Lightweight_rank', 'R_avg_TD_landed', 'R_Middleweight_rank', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'sig_str_dif', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'R_win_by_Decision_Split', 'age_dif', 'R_odds']
2


In [163]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [164]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 23281.749636121684
Feature: B_avg_SIG_STR_landed Score: 26541.9184083149
The best feature was B_avg_SIG_STR_landed.  It scored 26541.9184083149
Current best score is: 26541.9184083149
Feature: B_current_lose_streak Score: 27221.26999888067
The best feature was B_current_lose_streak.  It scored 27221.26999888067
Current best score is: 27221.26999888067
Feature: R_avg_SIG_STR_landed Score: 27230.621232089183
Feature: B_Heavyweight_rank Score: 27617.264562173263
The best feature was B_Heavyweight_rank.  It scored 27617.264562173263
Current best score is: 27617.264562173263
Feature: R_avg_SUB_ATT Score: 28163.6669545558
Feature: R_Weight_lbs Score: 28295.433822226787
The best feature was R_Weight_lbs.  It scored 28295.433822226787
Current best score is: 28295.433822226787
Feature: R_avg_SUB_ATT Score: 28649.940801090102
The best feature was R_avg_SUB_ATT.  It scored 28649.940801090102
Current best score is: 28649.940801090102
Feature: R_Heavyweight_rank Score: 28985.

(3146, 33)

(3146,)

(3146, 2)

(200, 33)

...

(200, 31)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-335,260,1.000000,3.474410e-13,0
1,-400,300,0.997221,2.779213e-03,0
2,-125,100,0.994324,5.676416e-03,0
3,-177,140,1.000000,5.197682e-12,0
4,-265,205,0.725852,2.741482e-01,0
...,...,...,...,...,...
195,100,-120,0.866068,1.339323e-01,0
196,-185,160,0.781503,2.184972e-01,0
197,-115,-105,0.983691,1.630911e-02,0
198,-175,155,0.999996,4.488425e-06,1



          Number of matches: 200
          Number of bets: 190
          Number of winning bets: 115
          Number of losing bets: 75
          Number of underdog bets: 67
          Number of underdog wins: 28
          Number of underdog losses: 39
          Number of Favorite bets: 113
          Number of favorite wins: 81
          Number of favorite losses: 32
          Number of even bets: 10
          Number of even wins: 6
          Number of even losses: 4
          Profit: 1224.4864765392913
          Profit per bet: 6.44466566599627
          Profit per match: 6.122432382696457
          
          
HI
HI
HI
HI
HI


In [165]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3243, 22)

(3243,)

(3243, 2)

(200, 22)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-335,260,1.529884e-07,1.000000,0
1,-400,300,9.793429e-01,0.020657,0
2,-125,100,9.918051e-01,0.008195,0
3,-177,140,6.214273e-09,1.000000,0
4,-265,205,6.899160e-01,0.310084,0
...,...,...,...,...,...
195,100,-120,6.308345e-01,0.369165,0
196,-185,160,6.459781e-01,0.354022,0
197,-115,-105,9.262048e-01,0.073795,0
198,-175,155,9.999955e-01,0.000005,1



          Number of matches: 200
          Number of bets: 185
          Number of winning bets: 118
          Number of losing bets: 67
          Number of underdog bets: 78
          Number of underdog wins: 37
          Number of underdog losses: 41
          Number of Favorite bets: 97
          Number of favorite wins: 75
          Number of favorite losses: 22
          Number of even bets: 10
          Number of even wins: 6
          Number of even losses: 4
          Profit: 3226.1573928816833
          Profit per bet: 17.438688610171262
          Profit per match: 16.130786964408415
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [166]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [167]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [168]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [169]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=572, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=91, max_features='sqrt',\n                           min_samples_leaf=17, n_estimators=140,\n                           random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  "LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.99330891532001